In [49]:
import json
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

In [53]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

def get_data(path): 
        merged_df = pd.read_csv(path)
        strings = merged_df['String']
        str_lst = strings.values

        vocab = merged_df['Title'].values
        identifier = merged_df['identifier']
        identifier_vocab = pd.DataFrame({'ID': identifier, 'Vocab': vocab})
        identifier_vocab = identifier_vocab.set_index('Vocab')['ID'].to_dict()
        return merged_df, str_lst, vocab, identifier_vocab, identifier


merged_df, str_lst, vocab, identifier_vocab, identifier = get_data('data/merged_data_for_AI.csv')


In [54]:
import pickle


with open("data/study_subjects_with_embeddings.pkl", "rb") as pkl_file:
    study_data_we = pickle.load(pkl_file)


with open("data/lifelong_learning_with_embeddings.pkl", "rb") as pkl_file:
    ll_data_we = pickle.load(pkl_file)

In [55]:
tmp_string = study_data_we["Betriebswirtschaftslehre"][0]["name"]["raw_content"]
#t_emb = study_data_we["Betriebswirtschaftslehre"][0]["name"]["embedding"]

KeyError: 0

In [58]:
video_embeddings = pd.read_csv('data/sentence_transformer_embedding.csv')
del video_embeddings['Unnamed: 0']
video_embeddings_values = video_embeddings.values
print(video_embeddings_values.shape)

(71980, 768)


In [59]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(A, b):
    # Convert A and b to numpy arrays
    A = np.array(A)
    b = np.array(b)

    # Calculate cosine similarity between A and b
    similarity_scores = cosine_similarity(A, b.reshape(1, -1)).flatten()

    # Sort the indices based on similarity scores
    sorted_indices = np.argsort(similarity_scores)[::-1]

    return sorted_indices


In [60]:
res= calculate_cosine_similarity(video_embeddings_values,t_emb)
print(res)

[47039 34064 64229 ...  8116 42328 45444]


In [61]:
identifier[47039]

'PHe0bXAIuk0'

In [83]:
def get_top_n_videos(subject,lj_data, video_data_embeddings, video_titles, video_identifier, n = 3):
    subject_dict = lj_data[subject]
    print("Learning journey for ", subject, " is:")
    for i in subject_dict.keys():
        
        print(i)
        tmp_string = subject_dict[i]["name"]["raw_content"]
        t_emb = subject_dict[i]["name"]["embedding"]
        print("to learn ", tmp_string, " watch the following videos:")
        cosine_similarity_sorted = calculate_cosine_similarity(video_data_embeddings,t_emb)
        for index, v in enumerate(cosine_similarity_sorted[:3]):
            title = video_titles[v]
            #print(index)
            #print(title)
            url = "https://www.youtube.com/watch?v=" + video_identifier[v]
            print("Video %i: %s" % (index + 1, title))
            print(url)


In [63]:
study_data_we.keys()

dict_keys(['Betriebswirtschaftslehre', 'Informatik', 'Medizin', 'Maschinenbau', 'Elektrotechnik', 'Psychologie', 'Jura/Rechtswissenschaften', 'Architektur', 'Chemie', 'Biologie', 'Geschichte', 'Soziologie', 'Volkswirtschaftslehre', 'Mathematik', 'Physik', 'Politikwissenschaft', 'Medienwissenschaft', 'Sprachwissenschaft/Linguistik', 'Pädagogik', 'Philosophie', 'Kunstgeschichte', 'Sportwissenschaft', 'Geografie', 'Musikwissenschaft', 'Ethnologie'])

In [78]:
#study_data_we["Informatik"]["Grundlagen der Informatik"]

In [86]:
study_data_we.keys()

dict_keys(['Betriebswirtschaftslehre', 'Informatik', 'Medizin', 'Maschinenbau', 'Elektrotechnik', 'Psychologie', 'Jura/Rechtswissenschaften', 'Architektur', 'Chemie', 'Biologie', 'Geschichte', 'Soziologie', 'Volkswirtschaftslehre', 'Mathematik', 'Physik', 'Politikwissenschaft', 'Medienwissenschaft', 'Sprachwissenschaft/Linguistik', 'Pädagogik', 'Philosophie', 'Kunstgeschichte', 'Sportwissenschaft', 'Geografie', 'Musikwissenschaft', 'Ethnologie'])

In [85]:
get_top_n_videos("Betriebswirtschaftslehre", study_data_we, video_embeddings_values,merged_df["Title"],merged_df["identifier"])

Learning journey for  Betriebswirtschaftslehre  is:
Grundlagen der Betriebswirtschaftslehre
to learn  Grundlagen der Betriebswirtschaftslehre  watch the following videos:
Video 1: How The Economic Machine Works by Ray Dalio
https://www.youtube.com/watch?v=PHe0bXAIuk0
Video 2: The Theory of Production - Economics A2 Level Unit 3
https://www.youtube.com/watch?v=jph_CTpjNAY
Video 3: 3 Economic Rules of Thumb
https://www.youtube.com/watch?v=X0KOV0o1P30
Marketing
to learn  Marketing  watch the following videos:
Video 1: What is Marketing? Marketing In The Real and Business World - Marketing 101
https://www.youtube.com/watch?v=VzsNqYARnd0
Video 2: How to Develop a Marketing Strategy by Philip Kotler
https://www.youtube.com/watch?v=ghFwpoH71NM
Video 3: Marketing Mix: Promotion Strategy part 2
https://www.youtube.com/watch?v=FT2mJBbU-wE
Rechnungswesen
to learn  Rechnungswesen  watch the following videos:
Video 1: Introduction to Modern Internal Audit (Take Quiz in Description to get 1 CPE)
htt

In [87]:
ll_data_we.keys()

dict_keys(['Barista', 'Gärtner', 'Stricken', 'Basteln', 'Malen', 'Kommunizieren', 'Meditation', 'Campingmanagement', 'Scrum', 'Azure DevOps', 'Projektmanagement', 'Immobilien', 'Teamleitung'])

In [89]:
get_top_n_videos("Azure DevOps", ll_data_we, video_embeddings_values,merged_df["Title"],merged_df["identifier"])

Learning journey for  Azure DevOps  is:
Grundlagen der Azure Cloud
to learn  Grundlagen der Azure Cloud  watch the following videos:
Video 1: Cloud Computing Fundamentals
https://www.youtube.com/watch?v=uroryFU78gM
Video 2: Intro to Google Cloud, a song!
https://www.youtube.com/watch?v=iVFmlgWvJcg
Video 3: Continuous Integration, Continuous Deployment (CI-CD) with Azure DevOps
https://www.youtube.com/watch?v=jRgLSMlp28U
Verwalten von Source Control
to learn  Verwalten von Source Control  watch the following videos:
Video 1: HikCentral Access Control - Access Control Module Setup and Management
https://www.youtube.com/watch?v=nezbYEC0aec
Video 2: Track 2 01 From Workstation to Domain Admin Why Secure Administration Isnt Secure and How to Fix It
https://www.youtube.com/watch?v=Wdbm2_1tn14
Video 3: Control Techniques F600 Pump Drive Energy Features
https://www.youtube.com/watch?v=4LicbciyAbc
Azure Boards
to learn  Azure Boards  watch the following videos:
Video 1: Azure Arc enabled Kubern